In [ ]:
import pandas as pd
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

## Custom libraries
import index_helpers as ih

### Import and prep data

In [ ]:
## Import, index, and split
df = ih.read_and_merge_segmented_data()
df = ih.index_df_by_person(df)
df = ih.categorical_float_to_int(df)
df = ih.categorical_to_dummy(df)
X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df.drop("Label", axis=1),
                                                             label = df["Label"])

## Modify data for GroupKFold
groups = y_train.reset_index()['File_Name_split']
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

### Create pipeline and fit classifier

In [ ]:
## Make pipeline - name classifier "clf"
clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("clf", xgb.XGBClassifier())])

## Use "clf__" in order to correctly assign parameters to the clf object
clf_param_grid = {'clf__min_child_weight': [0.1, 1, 5, 10, 50],
                  'clf__gamma': [0.5, 1, 1.5, 2, 5],
                  'clf__subsample': [0.6, 0.8, 1.0],
                  'clf__colsample_bytree': [0.6, 0.8, 1.0],
                  'clf__max_depth': [5, 10, 25, 50, 75],
                  'clf__learning_rate': [0.0001, 0.001, 0.1, 1],
                  'clf__n_estimators': [50, 100, 250, 500],
                  'clf__reg_alpha': [0.0001, 0.001, 0.1, 1],
                  'clf__reg_lambda': [0.0001, 0.001, 0.1, 1]}

## Instantiate GroupKFold to avoid data leakage - to be passed to cv
gkf=GroupKFold(n_splits=10)

## Set up Randomized search CV
clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                  param_distributions=clf_param_grid,
                                  cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=50)

## Perform Group K-Cross-validation
clf_rand_auc.fit(X_train, y_train, groups=groups)

## Print results
print("Best score: ",  clf_rand_auc.best_score_)
print("Best estimator: ", clf_rand_auc.best_estimator_)